In [1]:
import json

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [3]:
import datetime, seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_excel('House_Rent_Train.xlsx')

In [5]:
df = pd.DataFrame(df)

In [6]:
df

,id,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,...,bathroom,facing,cup_board,floor,total_floor,amenities,water_supply,building_type,balconies,rent
0,ff8081815733a243015733b2876600a6,BHK2,NaN,21-04-2018 14:44,12.934471,77.634471,FAMILY,1,1,1,...,2.0,E,2.0,6.0,12.0,"{""LIFT"":true,""GYM"":true,""INTERNET"":true,""AC"":f...",CORP_BORE,AP,2.0,40000.0
1,ff8081815ee25e15015ee50004da2acd,BHK2,Bellandur,2017-09-10 12:48:00,12.929557,77.672280,ANYONE,0,1,0,...,2.0,NE,2.0,3.0,4.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",CORPORATION,AP,2.0,22000.0
2,ff80818163f29c9e0163f46770b873e7,BHK3,Thiruvanmiyur,2018-12-06 22:02:00,12.982870,80.262012,FAMILY,0,1,0,...,3.0,E,3.0,1.0,5.0,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",CORP_BORE,AP,3.0,28000.0
3,ff80818164b68a700164b74b1b247a1d,BHK1,Attiguppe,20-07-2018 16:22,12.955991,77.531634,FAMILY,0,0,0,...,1.0,E,1.0,1.0,2.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORPORATION,IH,0.0,8000.0
4,ff80818163b1faf00163b4b9b8a163b1,BHK3,Kodihalli,31-05-2018 17:41,12.963903,77.649446,FAMILY,0,0,0,...,3.0,E,4.0,0.0,0.0,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",CORPORATION,IH,1.0,45000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20550,19,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20551,20,BHK2,NaN,NaN,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20552,21,BHK2,NaN,NaN,NaN,NaN,NaN,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20553,22,BHK2,NaN,NaN,NaN,NaN,NaN,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.isna().sum()

id                   0
type                27
locality           160
activation_date     23
latitude            23
longitude           23
lease_type          23
gym                  0
lift                 0
swimming_pool        0
negotiable           0
furnishing           0
parking              0
property_size        0
property_age        23
bathroom            23
facing              23
cup_board           23
floor               23
total_floor         23
amenities           23
water_supply        23
building_type       23
balconies           23
rent                23
dtype: int64

In [8]:
df['activation_date']= pd.to_datetime(df['activation_date'])

### Here amenities is a dictionery like column.. so to fill missing data I'll at first enter '5' for the missing values..

In [9]:
df['amenities']=df['amenities'].fillna('5')

### Here I'll convert it into a json files..

In [10]:
df['amenities']=df['amenities'].apply(json.loads)

### Now will divide the data:

In [11]:
miss = df[df['amenities']==5]

In [12]:
mis = df[df.index.isin(miss.index)]
com = df[~df.index.isin(miss.index)]

In [13]:
com['amenities'].isna().sum()

0

In [14]:
com['amenities']

0        {'LIFT': True, 'GYM': True, 'INTERNET': True, ...
1        {'LIFT': True, 'GYM': False, 'INTERNET': False...
2        {'LIFT': True, 'GYM': False, 'INTERNET': False...
3        {'LIFT': False, 'GYM': False, 'INTERNET': Fals...
4        {'LIFT': False, 'GYM': False, 'INTERNET': Fals...
                               ...                        
20527    {'LIFT': True, 'GYM': True, 'INTERNET': True, ...
20528    {'LIFT': False, 'GYM': False, 'INTERNET': True...
20529    {'LIFT': True, 'GYM': True, 'INTERNET': True, ...
20530    {'LIFT': True, 'GYM': False, 'INTERNET': True,...
20531    {'LIFT': False, 'GYM': False, 'INTERNET': Fals...
Name: amenities, Length: 20532, dtype: object

### Now, I'll make diff columns for every amenity in the column 'amenities'

In [15]:
amenities_list = ['LIFT', 'GYM', 'INTERNET', 'AC', 'CLUB', 'INTERCOM', 'POOL', 'CPA', 'FS', 'SERVANT', 'SECURITY', 'SC', 'GP', 'PARK', 'RWH', 'STP', 'HK', 'PB', 'VP']
for amenity in amenities_list:
    com[amenity] = com['amenities'].apply(lambda x: 1 if x.get(amenity, False) else 0)

C:\Users\DELL\AppData\Local\Temp\ipykernel_13640\1722050686.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com[amenity] = com['amenities'].apply(lambda x: 1 if x.get(amenity, False) else 0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_13640\1722050686.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com[amenity] = com['amenities'].apply(lambda x: 1 if x.get(amenity, False) else 0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_13640\1722050686.py:3: SettingWithCopyWarning: 
A value is trying to be set on 

In [16]:
com.columns, mis.columns

(Index(['id', 'type', 'locality', 'activation_date', 'latitude', 'longitude',
        'lease_type', 'gym', 'lift', 'swimming_pool', 'negotiable',
        'furnishing', 'parking', 'property_size', 'property_age', 'bathroom',
        'facing', 'cup_board', 'floor', 'total_floor', 'amenities',
        'water_supply', 'building_type', 'balconies', 'rent', 'LIFT', 'GYM',
        'INTERNET', 'AC', 'CLUB', 'INTERCOM', 'POOL', 'CPA', 'FS', 'SERVANT',
        'SECURITY', 'SC', 'GP', 'PARK', 'RWH', 'STP', 'HK', 'PB', 'VP'],
       dtype='object'),
 Index(['id', 'type', 'locality', 'activation_date', 'latitude', 'longitude',
        'lease_type', 'gym', 'lift', 'swimming_pool', 'negotiable',
        'furnishing', 'parking', 'property_size', 'property_age', 'bathroom',
        'facing', 'cup_board', 'floor', 'total_floor', 'amenities',
        'water_supply', 'building_type', 'balconies', 'rent'],
       dtype='object'))

### Here I'll add the extra columns in 'miss' dataset equal to the number of amenity columns in 'com' dataset.. all the columns will be consisting of '0'

In [17]:
mis_p = np.pad(mis,((0,0),(0,19)), constant_values=np.nan)

### Now I will stock(add) these two dataset 'col' & 'mis_p' vertically !!

In [18]:
data = np.vstack((com,mis_p))

In [19]:
data = pd.DataFrame(data)

In [20]:
data.columns

RangeIndex(start=0, stop=44, step=1)

In [21]:
data

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,ff8081815733a243015733b2876600a6,BHK2,NaN,2018-04-21 14:44:00,12.934471,77.634471,FAMILY,1,1,1,...,0,1,1,1,0,1,1,0,1,1
1,ff8081815ee25e15015ee50004da2acd,BHK2,Bellandur,2017-09-10 12:48:00,12.929557,77.67228,ANYONE,0,1,0,...,0,1,1,0,1,1,1,0,1,1
2,ff80818163f29c9e0163f46770b873e7,BHK3,Thiruvanmiyur,2018-12-06 22:02:00,12.98287,80.262012,FAMILY,0,1,0,...,0,0,1,0,1,0,0,0,1,1
3,ff80818164b68a700164b74b1b247a1d,BHK1,Attiguppe,2018-07-20 16:22:00,12.955991,77.531634,FAMILY,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ff80818163b1faf00163b4b9b8a163b1,BHK3,Kodihalli,2018-05-31 17:41:00,12.963903,77.649446,FAMILY,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20550,19,NaN,NaN,NaT,NaN,NaN,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20551,20,BHK2,NaN,NaT,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20552,21,BHK2,NaN,NaT,NaN,NaN,NaN,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20553,22,BHK2,NaN,NaT,NaN,NaN,NaN,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now we have to change the column name or it will be a problem later on..

In [22]:
col_name = {'0': 'id', '1': 'type', '2': 'locality', '3': 'activation_date', '4': 'latitude',
    '5': 'longitude', '6': 'lease_type', '7': 'gym', '8': 'lift', '9': 'swimming_pool',
    '10': 'negotiable', '11': 'furnishing', '12': 'parking', '13': 'property_size',
    '14': 'property_age', '15': 'bathroom', '16': 'facing', '17': 'cup_board', '18': 'floor',
    '19': 'total_floor', '20': 'amenities', '21': 'water_supply', '22': 'building_type',
    '23': 'balconies', '24': 'rent', '25': 'LIFT', '26': 'GYM', '27': 'INTERNET', '28': 'AC',
    '29': 'CLUB', '30': 'INTERCOM', '31': 'POOL', '32': 'CPA', '33': 'FS', '34': 'SERVANT',
    '35': 'SECURITY', '36': 'SC', '37': 'GP', '38': 'PARK', '39': 'RWH', '40': 'STP', '41': 'HK',
    '42': 'PB', '43': 'VP'}

In [23]:
data.columns = data.columns.astype(str)

In [24]:
data.rename(columns=col_name, inplace=True)

### Changing the 'activation_date' column into days !!

In [25]:
data['age'] = (datetime.datetime.now() - data['activation_date']).dt.days

In [26]:
data

,id,type,locality,activation_date,latitude,longitude,lease_type,gym,lift,swimming_pool,...,SECURITY,SC,GP,PARK,RWH,STP,HK,PB,VP,age
0,ff8081815733a243015733b2876600a6,BHK2,NaN,2018-04-21 14:44:00,12.934471,77.634471,FAMILY,1,1,1,...,1,1,1,0,1,1,0,1,1,2039.0
1,ff8081815ee25e15015ee50004da2acd,BHK2,Bellandur,2017-09-10 12:48:00,12.929557,77.67228,ANYONE,0,1,0,...,1,1,0,1,1,1,0,1,1,2262.0
2,ff80818163f29c9e0163f46770b873e7,BHK3,Thiruvanmiyur,2018-12-06 22:02:00,12.98287,80.262012,FAMILY,0,1,0,...,0,1,0,1,0,0,0,1,1,1810.0
3,ff80818164b68a700164b74b1b247a1d,BHK1,Attiguppe,2018-07-20 16:22:00,12.955991,77.531634,FAMILY,0,0,0,...,0,0,0,0,0,0,0,0,0,1949.0
4,ff80818163b1faf00163b4b9b8a163b1,BHK3,Kodihalli,2018-05-31 17:41:00,12.963903,77.649446,FAMILY,0,0,0,...,0,0,0,1,0,0,0,0,1,1999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20550,19,NaN,NaN,NaT,NaN,NaN,NaN,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20551,20,BHK2,NaN,NaT,NaN,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20552,21,BHK2,NaN,NaT,NaN,NaN,NaN,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20553,22,BHK2,NaN,NaT,NaN,NaN,NaN,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data['rent'].isna().sum()

23

In [28]:
nums = [ 'latitude', 'longitude', 'bathroom','property_size', 'property_age', 'cup_board','floor', 'total_floor', 'balconies','rent',]

### Now for the other missing values I will fill them with midian or mode of the respective column.. (midian if numerical type & mode if categorical type)

In [30]:
for num in nums :
    data[num].fillna(data[num].median(), inplace=True)
rests = set(data.columns) - set(nums)
for rest in rests :
    data[rest].fillna(data[rest].mode()[0], inplace=True)

In [31]:
data.columns

Index(['id', 'type', 'locality', 'activation_date', 'latitude', 'longitude',
       'lease_type', 'gym', 'lift', 'swimming_pool', 'negotiable',
       'furnishing', 'parking', 'property_size', 'property_age', 'bathroom',
       'facing', 'cup_board', 'floor', 'total_floor', 'amenities',
       'water_supply', 'building_type', 'balconies', 'rent', 'LIFT', 'GYM',
       'INTERNET', 'AC', 'CLUB', 'INTERCOM', 'POOL', 'CPA', 'FS', 'SERVANT',
       'SECURITY', 'SC', 'GP', 'PARK', 'RWH', 'STP', 'HK', 'PB', 'VP', 'age'],
      dtype='object')

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20555 entries, 0 to 20554
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               20555 non-null  object        
 1   type             20555 non-null  object        
 2   locality         20555 non-null  object        
 3   activation_date  20555 non-null  datetime64[ns]
 4   latitude         20555 non-null  float64       
 5   longitude        20555 non-null  float64       
 6   lease_type       20555 non-null  object        
 7   gym              20555 non-null  object        
 8   lift             20555 non-null  object        
 9   swimming_pool    20555 non-null  object        
 10  negotiable       20555 non-null  object        
 11  furnishing       20555 non-null  object        
 12  parking          20555 non-null  object        
 13  property_size    20555 non-null  object        
 14  property_age     20555 non-null  float

In [33]:
for col in ['negotiable', 'property_size'] :
    data[col] = data[col].astype(np.int64)

In [34]:
data.to_excel('Preprocessed Data.xlsx')